In [1]:
from deepview import DeepView
import matplotlib.pyplot as plt
import numpy as np
import time
# ---------------------------
import demo_utils as demo

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import pandas as pd
from tqdm import tqdm
import torch_dct
from deepview.embeddings import init_umap, init_inv_umap
from deepview.fisher_metric import calculate_fisher
import os
import matplotlib as mpl
import warnings

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
%matplotlib qt

#### 流量前期处理

In [4]:
class DNN_NIDS(nn.Module):
    def __init__(self):
        super(DNN_NIDS, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(59, 4096),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(4096, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 9)
        )

    def forward(self, x):
        return self.model(x)

model=DNN_NIDS()
model.load_state_dict(torch.load('CIC_model2.pth'))
model.eval()

DNN_NIDS(
  (model): Sequential(
    (0): Linear(in_features=59, out_features=4096, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=4096, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=512, out_features=9, bias=True)
  )
)

In [5]:
class NIDS_dataset(Dataset):
    def __init__(self, file):
        df = pd.read_csv(file, index_col=False)
        self.items = np.array(df.iloc[:,:-1])
        self.items = (self.items - np.mean(self.items, axis=1, keepdims=True)) / (
                    np.std(self.items, axis=1, keepdims=True) + 0.00001)
        self.label = np.array(df.iloc[:,-1])

    def __len__(self):
        # print('item的长度是',len(self.items))
        return len(self.items)

    def __getitem__(self, idx):
        # print(self.items[idx,:],self.label[idx])
        return self.items[idx, :], self.label[idx]

In [37]:
label_list = {'BENIGN': 0,
              'PortScan': 1,
                'Web': 2,
                'SSH-Patator': 3,
                'FTP-Patator': 4,
                'Bot': 5,
                'DoS Hulk': 6,
                'DoS GoldenEye': 7,
                'DDoS': 8}

label_index = {}
dict_label = {}
for key in label_list.keys():
    dict_label[label_list[key]] = key
    label_index[key] = []
print(label_index)
print(dict_label)



{'BENIGN': [], 'PortScan': [], 'Web': [], 'SSH-Patator': [], 'FTP-Patator': [], 'Bot': [], 'DoS Hulk': [], 'DoS GoldenEye': [], 'DDoS': []}
{0: 'BENIGN', 1: 'PortScan', 2: 'Web', 3: 'SSH-Patator', 4: 'FTP-Patator', 5: 'Bot', 6: 'DoS Hulk', 7: 'DoS GoldenEye', 8: 'DDoS'}


In [38]:
data_dir = "CICdata.csv"
labels = {}
df = pd.read_csv(data_dir)
print(df.shape[0])
 

1601232


In [40]:

for i in range(df.shape[0]):
    label_numb = df.iloc[i,-1]
    label_index[dict_label[label_numb]].append(i) 

    

In [48]:
# train_index = []
# test_index = []
# for key in label_index.keys():
#     key_index = np.array(label_index[key])
#     data_num = key_index.shape[0]
#     if data_num > 100000:
#         data_num = 100000
#     train_num = data_num*0.9    
#     result = np.random.choice(key_index, size=int(train_num), replace=False).tolist()
#     train_index += result
#     test_index += [x for x in key_index.tolist() if x not in result]

# train_data = df.iloc[train_index,:]
# test_data = df.iloc[test_index,:]
# train_data.to_csv('train_data.csv',index=False)
# test_data.to_csv('test_data.csv',index=False)

In [163]:
index = []
for key in label_index.keys():
    key_index = np.array(label_index[key])
    result = np.random.choice(key_index, size=100, replace=False)
    index += result.tolist()
print(len(index))

900


In [164]:
select_data = df.iloc[index,:]
print(select_data.shape)

(900, 60)


In [56]:
# select_data.to_csv('select_data2.csv', index=False)
ids_data = NIDS_dataset('select_data.csv')
data = torch.tensor(ids_data.items).to(torch.float32)
label = torch.tensor(ids_data.label)
pred = torch.max(model(data),1)[1]
correct = (pred == label).sum()

num = pred.shape[0]
print(correct/num)

tensor(1.)


In [44]:
a = torch.nonzero(pred == label)
for t in a:
    print(t)
    break

tensor([7])


In [20]:
correct_index = []
for i in range(data.shape[0]):
    sample = data[i].reshape(1,-1)
    sample_label = label[i]
    pred = torch.max(model(sample),1)[1]
    if sample_label == pred:
        correct_index.append(i)

data_dir = "select_data.csv"
df = pd.read_csv(data_dir)
df = df.iloc[correct_index,:]
df.shape

(875, 60)

In [21]:
df.to_csv('select_data.csv', index=False)

In [31]:
ids_data = NIDS_dataset('select_data.csv')
data = torch.tensor(ids_data.items).to(torch.float32)
label = torch.tensor(ids_data.label)

In [54]:
model=DNN_NIDS()
model.load_state_dict(torch.load('CIC_model2.pth'))
model.eval()

DNN_NIDS(
  (model): Sequential(
    (0): Linear(in_features=59, out_features=4096, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=4096, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=512, out_features=9, bias=True)
  )
)

In [55]:

pred = torch.max(model(data),1)[1]
correct = (pred == label).sum()
num = pred.shape[0]
print(correct/num)

tensor(0.2400)


### deepview降维

In [16]:
class deepView:
    def __init__(self, pred_fn, classes, max_samples, batch_size, data_shape, n=5, 
				 lam=0.65, resolution=100, cmap='tab10', interactive=True, verbose=True,
				 title='CICIDS2017', data_viz=None, mapper=None, inv_mapper=None, **kwargs):
        """ 
         pred_fn: 该函数就是调用分类模型，输入数据，输出分类预测概率
         classes: 所有类别名
         max_samples: 可视化保留的最大样本数据，
                     如果通过 "add_samples"函数传入的数据样本数超过此限制
                     则会先删除最老的样本。
         n: int型 计算两个样本距离用到的插值次数
		 lam: 将欧几里得度量与基于分类的判别距离进行加权的系数
        
        """
        self.model = pred_fn
        self.classes = classes
        self.n_classes = len(classes)
        self.max_samples = max_samples
        self.batch_size = batch_size
        self.data_shape = data_shape
        self.n = n
        self.lam = lam
        self.resolution = resolution
        self.cmap = plt.get_cmap(cmap)
        self.discr_distances = np.array([])
        self.eucl_distances = np.array([])
        self.samples = np.empty([0, data_shape])
        self.embedded = np.empty([0, 2])
        self.y_true = np.array([])
        self.y_pred = np.array([])
        self.classifier_view = np.array([])
        self.verbose = verbose
        self.interactive = interactive
        self.title = title
        self.data_viz = data_viz
        self._init_mappers(mapper, inv_mapper, kwargs)
    

    @property
    def distances(self):
        '''
        Combines euclidian with discriminative fisher distances.
        Here the two distance measures are weighted with lambda
        to emphasise structural properties (lambda > 0.5) or
        to emphasise prediction properties (lambda < 0.5).
        '''
        eucl_scale = 1. / self.eucl_distances.max()
        fisher_scale = 1. / self.discr_distances.max()
        eucl = self.eucl_distances * eucl_scale * self.lam
        fisher = self.discr_distances * fisher_scale * (1.-self.lam)
        stacked = np.dstack((fisher, eucl))
        return stacked.sum(-1)

    @property
    def num_samples(self):
        '''
        Returns the number of samples in DeepView.
        '''
        return len(self.samples)
    
    def reset(self):
        '''
        Resets the state of DeepView to the point of initialization.
        '''
        self.discr_distances = np.array([])
        self.eucl_distances = np.array([])
        self.samples = np.empty([0, self.data_shape])
        self.embedded = np.empty([0, 2])
        self.y_true = np.array([])
        self.y_pred = np.array([])
        self.classifier_view = np.array([])

    def close(self):
        '''
        Closes the matplotlib window, terminates DeepView.
        '''
        plt.close()

    def set_lambda(self, lam):
        '''
        Sets a new lambda and recomputes the embeddings and
        decision boundary grid.
        '''
        if self.lam == lam:
            return
        self.lam = lam
        self.update_mappings()
    
    def _init_mappers(self, mapper, inv_mapper, kwargs):
        if mapper is not None:
            self.mapper = mapper
        else:
            self.mapper = init_umap(kwargs)
        if inv_mapper is not None:
            self.inverse = inv_mapper
        else:
            self.inverse = init_inv_umap(kwargs)

    def _get_plot_measures(self):
        '''
        Computes the axis limits of the main plot by using
        min/max values of the 2D sample embedding and adding 10%
        on either side.
        '''
        ebd_min = np.min(self.embedded, axis=0)
        ebd_max = np.max(self.embedded, axis=0)
        ebd_extent = ebd_max - ebd_min

        # get extent of embedding
        x_min, y_min = ebd_min - 0.1 * ebd_extent
        x_max, y_max = ebd_max + 0.1 * ebd_extent
        return x_min, y_min, x_max, y_max

    def _predict_batches(self, x):
        '''
        Predicts an array of samples batchwise.
        '''
        n_inputs = len(x)
        preds = np.zeros([n_inputs, self.n_classes])

        for i in range(0, n_inputs, self.batch_size):
            n_preds = min(i + self.batch_size, n_inputs)
            batch = x[i:n_preds]
            preds[i:n_preds] = np.array(self.model(batch))

        return preds


    def add_samples(self, samples, labels):
        '''
        Adds samples points to the visualization.

        Parameters
        ----------
        samples : array-like
            List of new sample points [n_samples, *data_shape]
        labels : array-like
            List of labels for the sample points [n_samples, 1]
        '''
        # get predictions for the new samples
        Y_probs = self._predict_batches(samples)
        Y_preds = Y_probs.argmax(axis=1)
        # add new values to the DeepView lists
        self.queue_samples(samples, labels, Y_preds)

        # calculate new distances
        new_discr, new_eucl = calculate_fisher(self.model, samples, self.samples, 
            self.n, self.batch_size, self.n_classes, self.verbose)
        # add new distances
        self.discr_distances = self.update_matrix(self.discr_distances, new_discr)
        self.eucl_distances = self.update_matrix(self.eucl_distances, new_eucl)

        # update mappings
        self.update_mappings()
    

    def queue_samples(self, samples, labels, preds):
        '''
        Adds samples labels and predictions to the according lists of
        this deepview object. Old values will be discarded, when there are 
        more then max_samples.
        '''
        # add new samples and remove depricated samples
        self.samples = np.concatenate((samples, self.samples))[:self.max_samples]
        self.y_pred = np.concatenate((preds, self.y_pred))[:self.max_samples]
        self.y_true = np.concatenate((labels, self.y_true))[:self.max_samples]

    def update_matrix(self, old_matrix, new_values):
        '''
        When new distance values are calculated this merges old 
        and new distances into the same matrix.
        '''
        n_new = new_values.shape[0]
        n_keep = self.max_samples - n_new
        to_triu = np.triu(old_matrix, k=1)
        new_mat = np.zeros([self.num_samples, self.num_samples])
        new_mat[n_new:,n_new:] = to_triu[:n_keep,:n_keep]
        new_mat[:n_new] = new_values
        return new_mat + new_mat.transpose()

    def update_mappings(self):
        if self.verbose:
            print('Embedding samples ...')

        self.mapper.fit(self.distances)
        self.embedded = self.mapper.transform(self.distances)
        self.inverse.fit(self.embedded, self.samples)
        self.classifier_view = self.compute_grid()
    

    def compute_grid(self):
        '''
        Computes the visualisation of the decision boundaries.
        '''
        if self.verbose:
            print('Computing decision regions ...')
        # get extent of embedding
        x_min, y_min, x_max, y_max = self._get_plot_measures()
        # create grid
        xs = np.linspace(x_min, x_max, self.resolution)
        ys = np.linspace(y_min, y_max, self.resolution)
        self.grid = np.array(np.meshgrid(xs, ys))
        grid = np.swapaxes(self.grid.reshape(self.grid.shape[0],-1),0,1)

        # map gridmpoint to images
        grid_samples = self.inverse(grid)

        mesh_preds = self._predict_batches(grid_samples)
        mesh_preds = mesh_preds + 1e-8

        self.mesh_classes = mesh_preds.argmax(axis=1)
        mesh_max_class = max(self.mesh_classes)

        # get color of gridpoints
        color = self.cmap(self.mesh_classes/mesh_max_class)
        # scale colors by certainty
        h = -(mesh_preds*np.log(mesh_preds)).sum(axis=1)/np.log(self.n_classes)
        h = (h/h.max()).reshape(-1, 1)
        # adjust brightness
        h = np.clip(h*1.2, 0, 1)
        color = color[:,0:3]
        color = (1-h)*(0.5*color) + h*np.ones(color.shape, dtype=np.uint8)
        decision_view = color.reshape(self.resolution, self.resolution, 3)
        return decision_view
    

    def show_sample(self, event):
        '''
        Invoked when the user clicks on the plot. Determines the
        embedded or synthesised sample at the click location and 
        passes it to the data_viz method, together with the prediction, 
        if present a groun truth label and the 2D click location.
        '''

        # when there is an artist attribute, a 
        # concrete sample was clicked, otherwise
        # show the according synthesised image
        if hasattr(event, 'artist'):
            artist = event.artist
            ind = event.ind
            xs, ys = artist.get_data()
            point = [xs[ind][0], ys[ind][0]]
            sample, p, t = self.get_artist_sample(point)
            title = '%s <-> %s' if p != t else '%s --- %s'
            title = title % (self.classes[p], self.classes[t])
            self.disable_synth = True
        elif not self.disable_synth:
            # workaraound: inverse embedding needs more points
            # otherwise it doens't work --> [point]*5
            point = np.array([[ event.xdata , event.ydata ]]*5)

            # if the outside of the plot was clicked, points are None
            if None in point[0]:
                return

            sample = self.inverse(point)[0]
            sample += abs(sample.min())
            sample /= sample.max()
            title = 'Synthesised at [%.1f, %.1f]' % tuple(point[0])
            p, t = self.get_mesh_prediction_at(*point[0]), None
        else:
            self.disable_synth = False
            return

        is_image = self.is_image(sample)
        rank_perm = np.roll(range(len(sample.shape)), -1)
        sample_T = sample.transpose(rank_perm)
        is_transformed_image = self.is_image(sample_T)

        if self.data_viz is not None:
            self.data_viz(sample, point, p, t)
            return
        # try to show the image, if the shape allows it
        elif is_image:
            img = sample - sample.min()
        elif is_transformed_image:
            img = sample_T - sample_T.min()
        else:
            warnings.warn("Data visualization not possible, as the data points have"
                "no image shape. Pass a function in the data_viz argument,"
                "to enable custom data visualization.")
            return

        f, a = plt.subplots(1, 1)
        a.imshow(img / img.max())
        a.set_title(title)
          


    def _init_plots(self):
        '''
        Initialises matplotlib artists and plots.
        '''
        if self.interactive:
            plt.ion()
        self.fig, self.ax = plt.subplots(1, 1, figsize=(8, 8))
        # self.ax.set_title(self.title)
        self.desc = self.fig.text(0.5, 0.02, '', fontsize=8, ha='center')
        self.cls_plot = self.ax.imshow(np.zeros([5, 5, 3]), 
            interpolation='gaussian', zorder=0, vmin=0, vmax=1)

        self.sample_plots = []

        for c in range(self.n_classes):
            color = self.cmap(c/(self.n_classes-1))
            plot = self.ax.plot([], [], 'o', label=self.classes[c], 
                color=color, zorder=2, picker=mpl.rcParams['lines.markersize'])
            self.sample_plots.append(plot[0])

        for c in range(self.n_classes):
            color = self.cmap(c/(self.n_classes-1))
            plot = self.ax.plot([], [], 'o', markeredgecolor=color, 
                fillstyle='none', ms=12, mew=2.5, zorder=1)
            self.sample_plots.append(plot[0])

        # set the mouse-event listeners
        self.fig.canvas.mpl_connect('pick_event', self.show_sample)
        self.fig.canvas.mpl_connect('button_press_event', self.show_sample)
        self.disable_synth = False
        self.ax.legend()

    def show(self):
        '''
        Shows the current plot.
        '''
        if not hasattr(self, 'fig'):
            self._init_plots()

        x_min, y_min, x_max, y_max = self._get_plot_measures()

        self.cls_plot.set_data(self.classifier_view)
        self.cls_plot.set_extent((x_min, x_max, y_max, y_min))
        self.ax.set_xlim((x_min, x_max))
        self.ax.set_ylim((y_min, y_max))

        params_str = 'batch size: %d - n: %d - $\lambda$: %.2f - res: %d'
        desc = params_str % (self.batch_size, self.n, self.lam, self.resolution)
        self.desc.set_text(desc)

        for c in range(self.n_classes):
            data = self.embedded[self.y_true==c]
            print(data.shape)
            self.sample_plots[c].set_data(data.transpose())

        for c in range(self.n_classes):
            data = self.embedded[np.logical_and(self.y_pred==c, self.y_true!=c)]
            self.sample_plots[self.n_classes+c].set_data(data.transpose())

        if os.name == 'posix':
            self.fig.canvas.manager.window.raise_()
            
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        plt.show()
    

In [20]:
torch_model=DNN_NIDS()
torch_model.load_state_dict(torch.load('CIC_model2.pth'))
torch_model.eval()
ids_dataset = NIDS_dataset('select_data.csv')
softmax = torch.nn.Softmax(dim=-1)
def pred_wrapper(x):
    with torch.no_grad():
        x = np.array(x, dtype=np.float32)
        tensor = torch.from_numpy(x)
        logits = torch_model(tensor)
        probabilities = softmax(logits).cpu().numpy()
    return probabilities

def visualization(image, point2d, pred, label=None, title=None):
    f, a = plt.subplots()
    a.set_title(title)
    a.imshow(image.transpose([1, 2, 0]))


classes =  ('BENIGN',
            'PortScan',
            'Web',
            'SSH-Patator',
            'FTP-Patator',
            'Bot',
            'DoS Hulk',
            'DoS GoldenEye',
            'DDoS',
            )
batch_size = 512
max_samples = 1000
data_shape = (59)
lam = 0
title = 'Origin Samples'

deepview = deepView(pred_wrapper, classes, max_samples, batch_size, 
                    data_shape, lam=lam, title=title)



adv_datas = np.load('adv_data/adv_data.npy')
adv_labels = np.array([1]*adv_datas.shape[0])


In [21]:
# X = np.vstack((ids_dataset.items,adv_datas))
# Y = np.concatenate((ids_dataset.label,adv_labels))
# X = ids_dataset.items
X = adv_datas
Y = ids_dataset.label
deepview.add_samples(X,Y)

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...
Computing decision regions ...


In [22]:
deepview.show()

<ipython-input-16-385a92160010>:306: MatplotlibDeprecationWarning: Setting the line's pick radius via set_picker is deprecated since 3.3 and will be removed two minor releases later; use set_pickradius instead.
  plot = self.ax.plot([], [], 'o', label=self.classes[c],


(92, 2)
(100, 2)
(91, 2)
(100, 2)
(100, 2)
(99, 2)
(96, 2)
(100, 2)
(97, 2)


In [110]:
print(len(deepview.sample_plots))

18


In [15]:
print(len(deepview.y_pred))

919
